In [1]:
import warnings
import gc
import tensorflow as tf
from tensorflow import keras
from random import choice
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, GRU, Concatenate, Embedding, Flatten, Activation, Dropout
from sklearn.model_selection import KFold
from tensorflow.python.client import device_lib
warnings.filterwarnings('ignore')
import random

In [25]:
MAXLENGTH = 400
EMBEDDING_DIM = 128

In [26]:
FEATURES_SIZE = 39
CHAPTER_SIZE = 38
SUB_CHAPTER_SIZE = 223
QUESTION_SIZE = 1069

In [30]:
# 5 fold cross validation with LSTM-based model
X = np.array(grouped_data.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
val_losses = list()
val_aucs = list()
train_eval = list()
test_eval = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train_data_space = SPACE_DATASET(grouped_data[users_train], MAXLENGTH)
    val_data_space = SPACE_DATASET(grouped_data[users_val], MAXLENGTH)
    test_data_space = SPACE_DATASET(grouped_data[users_test], MAXLENGTH)
    #construct training input
    train_chapter=[]
    train_sub_chapter=[]
    train_question = []
    train_features=[]
    train_labels=[]
    for i in range(len(users_train)):
        user = train_data_space.__getitem__(i)
        train_chapter.append(user[0])
        train_sub_chapter.append(user[1]) 
        train_question.append(user[2])
        train_features.append(user[3])
        train_labels.append(user[4])
    train_chapter = np.array(train_chapter)
    train_sub_chapter = np.array(train_sub_chapter)
    train_question = np.array(train_question)
    train_features = np.array(train_features)
    train_labels= np.array(train_labels)[..., np.newaxis]

    #construct validation input
    val_chapter=[]
    val_sub_chapter=[]
    val_question = []
    val_features=[]
    val_labels=[]
    for i in range(len(users_val)):
        user = val_data_space.__getitem__(i)
        val_chapter.append(user[0])
        val_sub_chapter.append(user[1]) 
        val_question.append(user[2])
        val_features.append(user[3])
        val_labels.append(user[4])
    val_chapter = np.array(val_chapter)
    val_sub_chapter = np.array(val_sub_chapter)
    val_features = np.array(val_features)
    val_question = np.array(val_question)
    val_labels= np.array(val_labels)[..., np.newaxis]

    # construct test input
    test_chapter=[]
    test_sub_chapter=[]
    test_features=[]
    test_question=[]
    test_labels=[]
    for i in range(len(users_test)):
        user = test_data_space.__getitem__(i)
        test_chapter.append(user[0])
        test_sub_chapter.append(user[1]) 
        test_question.append(user[2])
        test_features.append(user[3])
        test_labels.append(user[4])
    test_chapter = np.array(test_chapter)
    test_sub_chapter = np.array(test_sub_chapter)
    test_features = np.array(test_features)
    test_question = np.array(test_question)
    test_labels= np.array(test_labels)[..., np.newaxis]

    # define loss function and evaluation metrics
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    acc = tf.keras.metrics.Accuracy()
    auc = tf.keras.metrics.AUC()

    def masked_bce(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return bce(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_acc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      flat_pred = (flat_pred >= 0.5)
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return acc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_auc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return auc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    # input layer
    input_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_sub_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_ques =  tf.keras.Input(shape=(MAXLENGTH))
    input_features = tf.keras.Input(shape=(MAXLENGTH, FEATURES_SIZE))

    # embedding layer for categorical features
    embedding_chap = Embedding(input_dim = CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_chap)
    embedding_sub_chap = Embedding(input_dim = SUB_CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_sub_chap) 
    embedding_ques = Embedding(input_dim = QUESTION_SIZE, output_dim = EMBEDDING_DIM)(input_ques)       
    # dense layer for numeric features
    dense_features = Dense(EMBEDDING_DIM,input_shape = (None, MAXLENGTH))(input_features)
    
    output = tf.concat([embedding_chap, embedding_sub_chap, embedding_ques, dense_features], axis = 2)

    pred = Dense(1, input_shape = (None, 4*EMBEDDING_DIM), activation='sigmoid')(output)

    model = tf.keras.Model(
        inputs=[input_chap, input_sub_chap,input_ques, input_features],
        outputs=pred,
        name='logistic_regression'
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt_adam = Adam(learning_rate = 0.005)
    model.compile(
        optimizer=opt_adam,
        loss= masked_bce,
        metrics = [masked_acc, masked_auc]
    )

    history = model.fit(
      [train_chapter, train_sub_chapter, train_question, train_features],
      train_labels,
      batch_size = 64,
      epochs = 100,
      validation_data=([val_chapter, val_sub_chapter, val_question, val_features], val_labels),
      callbacks=[callback]
    )
    val_losses.append(list(history.history['val_loss']))
    train_losses.append(list(history.history['loss']))
    val_aucs.append(list(history.history['val_masked_auc']))
    train_aucs.append(list(history.history['masked_auc']))
    train_score = model.evaluate([train_chapter, train_sub_chapter, train_question, train_features], train_labels)
    train_eval.append(train_score)
    test_score = model.evaluate([test_chapter, test_sub_chapter, test_question, test_features], test_labels)
    test_eval.append(test_score)
    print("Test: ", test_score)
    def reset_weights(model):
      for layer in model.layers: 
        if isinstance(layer, tf.keras.Model):
          reset_weights(layer)
          continue
        for k, initializer in layer.__dict__.items():
          if "initializer" not in k:
            continue
          # find the corresponding variable
          var = getattr(layer, k.replace("_initializer", ""))
          var.assign(initializer(var.shape, var.dtype))
    reset_weights(model)

Epoch 1/100
14/14 [==============================] - 4s 191ms/step - loss: 0.6039 - masked_acc: 0.5198 - masked_auc: 0.5440 - val_loss: 0.4833 - val_masked_acc: 0.6988 - val_masked_auc: 0.6914
Epoch 2/100
14/14 [==============================] - 2s 127ms/step - loss: 0.4551 - masked_acc: 0.7131 - masked_auc: 0.7201 - val_loss: 0.4620 - val_masked_acc: 0.7336 - val_masked_auc: 0.7545
Epoch 3/100
14/14 [==============================] - 2s 111ms/step - loss: 0.4422 - masked_acc: 0.7382 - masked_auc: 0.7629 - val_loss: 0.4587 - val_masked_acc: 0.7458 - val_masked_auc: 0.7756
Epoch 4/100
14/14 [==============================] - 2s 112ms/step - loss: 0.4368 - masked_acc: 0.7478 - masked_auc: 0.7796 - val_loss: 0.4551 - val_masked_acc: 0.7521 - val_masked_auc: 0.7869
Epoch 5/100
14/14 [==============================] - 2s 112ms/step - loss: 0.4360 - masked_acc: 0.7532 - masked_auc: 0.7892 - val_loss: 0.4559 - val_masked_acc: 0.7560 - val_masked_auc: 0.7938
Epoch 6/100
14/14 [================

In [31]:
t_eval = np.array(test_eval)
print("test avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("test avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("test avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

test avg loss:  0.43222416639328004 +/- 0.01181041006083645
test avg acc:  0.7703701615333557 +/- 0.0010545532951167036
test avg auc:  0.8164567589759827 +/- 0.0012246812515663559


In [32]:
t_eval = np.array(train_eval)
print("train avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("train avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("train avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

train avg loss:  0.43511608242988586 +/- 0.0008435815838008256
train avg acc:  0.7704720735549927 +/- 0.0011106196748008322
train avg auc:  0.8164590001106262 +/- 0.0012846862230395833
